**Step 0: Install Required Libraries**

In [ ]:
!pip install transformers datasets sentencepiece accelerate scikit-learn nltk torch tqdm numpy

**Step 1: Import Libraries and Download NLTK Data**

In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import nltk
from nltk.tokenize import word_tokenize
import re
import numpy as np
from tqdm import tqdm

nltk.download('punkt')

**Step 2: Define Data Loading and Preprocessing Functions**

In [ ]:
def load_and_preprocess_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    augmented_data = []
    for item in data:
        augmented_data.append(item)

        question = item['question']
        answer = item['answer']

        if question.lower().startswith('what is'):
            new_question = 'Could you tell me' + question[7:]
            augmented_data.append({'question': new_question, 'answer': answer})

        if not question.lower().startswith('can you'):
            new_question = f"Can you explain {question.lower()}"
            augmented_data.append({'question': new_question, 'answer': answer})

    return augmented_data


def create_dataset(data):
    questions = [item['question'] for item in data]
    answers = [item['answer'] for item in data]

    train_questions, test_questions, train_answers, test_answers = train_test_split(
        questions, answers, test_size=0.2, random_state=42
    )

    test_questions, val_questions, test_answers, val_answers = train_test_split(
        test_questions, test_answers, test_size=0.5, random_state=42
    )

    return {
        'train': Dataset.from_dict({'question': train_questions, 'answer': train_answers}),
        'test': Dataset.from_dict({'question': test_questions, 'answer': test_answers}),
        'validation': Dataset.from_dict({'question': val_questions, 'answer': val_answers})
    }

**Step 3: Define Model Training Functions**

In [ ]:
def preprocess_function(examples, tokenizer, max_length=384):
    inputs = [f"answer question: {q.strip()}" for q in examples['question']]
    targets = [a.strip() for a in examples['answer']]

    model_inputs = tokenizer(
        inputs,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    labels = tokenizer(
        targets,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

**Step 4: Define Custom Trainer with Metrics**

In [ ]:
class CustomTrainer(Trainer):
    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred

        decoded_preds = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        binary_preds = [1 if p == l else 0 for p, l in zip(decoded_preds, decoded_labels)]
        binary_labels = [1] * len(decoded_labels)

        return {
            'accuracy': accuracy_score(binary_labels, binary_preds),
            'f1': f1_score(binary_labels, binary_preds, average='binary'),
            'recall': recall_score(binary_labels, binary_preds, average='binary'),
            'precision': precision_score(binary_labels, binary_preds, average='binary')
        }

**Step 5: Define Model Training Function**

In [ ]:
def train_model(datasets, model_name="t5-base", output_dir="./results"):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    tokenized_datasets = {
        split: datasets[split].map(
            lambda x: preprocess_function(x, tokenizer),
            batched=True,
            remove_columns=datasets[split].column_names
        )
        for split in datasets.keys()
    }

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="steps",
        eval_steps=50,
        learning_rate=5e-5,
        per_device_train_batch_size=6,
        per_device_eval_batch_size=6,
        num_train_epochs=17,
        weight_decay=0.01,
        save_total_limit=2,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        gradient_accumulation_steps=2,
        warmup_steps=100,
        fp16=True
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    trainer.train()

    return model, tokenizer, trainer

**Step 6: Define Enhanced Chatbot Class**

In [ ]:
class EnhancedChatbot:
    def __init__(self, model_path):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)
        self.model = T5ForConditionalGeneration.from_pretrained(model_path).to(self.device)
        self.model.eval()
        self.fallback_response = "I'm sorry, I don't have information on that. Can you please rephrase or ask a different question?"
        self.predefined_responses = {
            "hi": "Hello! How can I assist you today?",
            "hello": "Hello! How can I assist you today?",
            "hey": "Hello! How can I assist you today?",
            "thanks": "You're welcome! Feel free to ask more questions.",
            "thank you": "You're welcome! Feel free to ask more questions.",
            "bye": "Goodbye! Have a great day!",
            "goodbye": "Goodbye! Have a great day!",
            "thankyou": "You're welcome! Feel free to ask more questions."
        }

    def preprocess_question(self, question):
        question = question.lower().strip()
        question = re.sub(r'[^a-z0-9\s?.!,]', '', question)
        if not question.endswith('?'):
            question += '?'
        return question

    def generate_response(self, question, max_length=256, num_beams=5, confidence_threshold=0.5):
        if question.lower() in self.predefined_responses:
            return self.predefined_responses[question.lower()]

        processed_question = self.preprocess_question(question)
        input_text = f"answer question: {processed_question}"

        inputs = self.tokenizer(
            input_text,
            return_tensors="pt",
            max_length=max_length,
            truncation=True,
            padding=True
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs["input_ids"],
                max_length=max_length,
                num_beams=num_beams,
                length_penalty=1.0,
                early_stopping=True,
                no_repeat_ngram_size=2,
                temperature=0.7,
                output_scores=True,
                return_dict_in_generate=True
            )

        logits = outputs.scores[0]
        probs = torch.softmax(logits, dim=-1)
        confidence = probs.max().item()

        response = self.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

        if confidence < confidence_threshold:
            return self.fallback_response

        return response

**Step 7: Evaluation Function**

In [ ]:
def evaluate_model(chatbot, test_dataset):
    predictions = []
    references = []

    for item in tqdm(test_dataset):
        question = item['question']
        reference = item['answer']
        prediction = chatbot.generate_response(question)

        predictions.append(prediction)
        references.append(reference)

    correct_predictions = [1 if p == r else 0 for p, r in zip(predictions, references)]
    binary_labels = [1] * len(references)

    metrics = {
        'accuracy': accuracy_score(binary_labels, correct_predictions),
        'f1': f1_score(binary_labels, correct_predictions, average='binary'),
        'recall': recall_score(binary_labels, correct_predictions, average='binary'),
        'precision': precision_score(binary_labels, correct_predictions, average='binary')
    }

    return metrics

**Step 8: Model Training and Evaluation**

In [ ]:
def train_and_evaluate():
    print("Loading and preprocessing data...")
    data = load_and_preprocess_data('LSPU-Student-Handbook-dataset.json')
    datasets = create_dataset(data)

    print("\nTraining model...")
    model, tokenizer, trainer = train_model(datasets)

    print("\nSaving model...")
    output_dir = "./enhanced-t5-chatbot"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    chatbot = EnhancedChatbot(output_dir)

    print("\nEvaluating model on test dataset...")
    metrics = evaluate_model(chatbot, datasets['test'])
    print("\nModel Metrics:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")

    return output_dir

**Step 9: Chatbot Interface**

In [ ]:
def run_chatbot(model_path):
    print("\nInitializing chatbot...")
    chatbot = EnhancedChatbot(model_path)

    print("\nLSPU Chatbot is ready! Type 'exit' or 'quit' to stop.")
    print("Type 'metrics' to see the model's performance metrics again.")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye! Thank you for using the LSPU Student Handbook Chatbot.")
            break

        if user_input.lower() == "metrics":
            data = load_and_preprocess_data('LSPU-Student-Handbook-dataset.json')
            datasets = create_dataset(data)
            metrics = evaluate_model(chatbot, datasets['test'])
            print("\nModel Metrics:")
            print(f"Accuracy: {metrics['accuracy']:.4f}")
            print(f"F1 Score: {metrics['f1']:.4f}")
            print(f"Recall: {metrics['recall']:.4f}")
            print(f"Precision: {metrics['precision']:.4f}")
            continue

        response = chatbot.generate_response(user_input)
        print(f"Bot: {response}")

**Step 10: Main Execution**

In [ ]:
def main():
    print("LSPU Student Handbook Chatbot System")
    print("====================================")

    model_path = "./enhanced-t5-chatbot"

    if not os.path.exists(model_path):
        print("Model not found. Training and saving the model...")
        model_path = train_and_evaluate()
    else:
        print("Loading pre-trained model...")

    run_chatbot(model_path)

if __name__ == "__main__":
    main()